In [1]:
#trigram model uses nltk only, the other imports are for GUI, file reading and symbols removal
import nltk
nltk.download('punkt')

import tkinter as tk
import customtkinter as ctk
import codecs
import re

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\sata\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
""" input_file = "raw_data.txt"
output_file = "sampled_data.txt"

with codecs.open(input_file, 'r', encoding='utf8', errors='ignore') as src_file:
        text = src_file.read().split()[:210000]

with codecs.open(output_file, 'w', encoding='utf8', errors='ignore') as output_file:
        output_file.write(" ".join(text)) """

' input_file = "raw_data.txt"\noutput_file = "sampled_data.txt"\n\nwith codecs.open(input_file, \'r\', encoding=\'utf8\', errors=\'ignore\') as src_file:\n        text = src_file.read().split()[:210000]\n\nwith codecs.open(output_file, \'w\', encoding=\'utf8\', errors=\'ignore\') as output_file:\n        output_file.write(" ".join(text)) '

In [3]:
# Define a function to train the trigram model on a text file
def train_model(file_path, no_words):
    # Read the text from the file
    with codecs.open(file_path, 'r', encoding='utf8', errors='ignore') as src_file:
        text = src_file.read()      
    
    #OPTIONAL remove symbols and punctuation
    text = re.sub('[^a-zA-Z0-9]', ' ', text)
    
    # Tokenize the text into words
    words = nltk.word_tokenize(text)

    # Create a list of trigrams from the words
    trigrams = list(nltk.trigrams(words))

    # Create a dictionary to store the trigram frequencies
    trigram_freqs = {}


    # Loop over the trigrams and count their frequencies
    for trigram in trigrams:
        prefix = trigram[:2]
        suffix = trigram[2]
        if prefix in trigram_freqs:
            if suffix in trigram_freqs[prefix]:
                trigram_freqs[prefix][suffix] += 1
            else:
                trigram_freqs[prefix][suffix] = 1
        else:
            trigram_freqs[prefix] = {suffix: 1}

    # Convert the trigram frequencies to probabilities
    trigram_probs = {}
    for prefix in trigram_freqs:
        total_count = sum(trigram_freqs[prefix].values())
        trigram_probs[prefix] = {suffix: count / total_count for suffix, count in trigram_freqs[prefix].items()}

    # Return the trigram probabilities
    return trigram_probs

In [4]:
# Define a function to suggest word completions based on the trigram model
def suggest_completions(trigram_probs, input_str):
    # Tokenize the input string into words
    words = nltk.word_tokenize(input_str)

    # Get the last two words
    prefix = tuple(words[-2:])

    # Get the probabilities of the next words
    if prefix in trigram_probs:
        probs = trigram_probs[prefix]
    else:
        probs = {}

    # Sort the probabilities in descending order
    sorted_probs = sorted(probs.items(), key=lambda x: x[1], reverse=True)

    # Get the top 5 suggestions
    suggestions = [suffix for suffix,_ in sorted_probs[:5]]

    print(sorted_probs)
    # Return the suggestions
    return suggestions

In [5]:
def suggestOneWordInput(trigram_probs, input_str):
    
     # Tokenize the input string into words
    words = nltk.word_tokenize(input_str)
    
    # Get the last prefix
    prefix = (words[-1],)
    
    # Get the probabilities of the next words
    completions = []
    for k, v in trigram_probs.items():
        if k[1] == prefix[0]:
            completions.extend([(k[0], x, y) for x, y in v.items()])

    # Sort the completions in descending order
    sorted_completions = sorted(completions, key=lambda x: x[2], reverse=True)

    # Get the top 5 suggestions
    suggestions = [x for x, _, _ in sorted_completions[:5]]

    # Return the suggestions
    return suggestions

In [6]:
# Define a function to update the suggestions in the GUI
def update_suggestions():
    input_str = input_box.get()
    checkInput = nltk.word_tokenize(input_str)
    if len(checkInput) > 1:
        suggestions = suggest_completions(trigram_probs, input_str)
    elif len(checkInput) == 1:
        suggestions = suggestOneWordInput(trigram_probs, input_str)
    else:
        suggestions = []
    suggestion_str = '\n'.join(suggestions)
    suggestion_label.configure(text=suggestion_str)

In [7]:
# Load the text file and train the trigram model
file_path = 'sampled_data.txt'
trigram_probs = train_model(file_path, 200000)

In [8]:
# Create the GUI
ctk.set_appearance_mode("System")
ctk.set_default_color_theme("blue")

app = ctk.CTk()
app.geometry("720x480")
app.title("Autofill")

title = ctk.CTkLabel(app, text="Enter text")
title.pack(padx=10, pady=10)

# Create the input box
user_input = tk.StringVar()
input_box = ctk.CTkEntry(app, width=350, height=40, textvariable=user_input)
input_box.pack(pady=20)
input_box.bind('<KeyRelease>', lambda event: update_suggestions())

# Create the suggestion label
suggestion_label = ctk.CTkLabel(app, text='', font=('Arial', 18))
suggestion_label.pack()

app.mainloop()


[]
[]
[]
[]
[('City', 0.15384615384615385), ('Times', 0.1282051282051282), ('Review', 0.07692307692307693), ('and', 0.05128205128205128), ('Stock', 0.05128205128205128), ('police', 0.02564102564102564), ('She', 0.02564102564102564), ('The', 0.02564102564102564), ('or', 0.02564102564102564), ('magazine', 0.02564102564102564), ('against', 0.02564102564102564), ('ICE', 0.02564102564102564), ('from', 0.02564102564102564), ('the', 0.02564102564102564), ('a', 0.02564102564102564), ('If', 0.02564102564102564), ('marks', 0.02564102564102564), ('who', 0.02564102564102564), ('will', 0.02564102564102564), ('Brad', 0.02564102564102564), ('to', 0.02564102564102564), ('court', 0.02564102564102564), ('where', 0.02564102564102564), ('s', 0.02564102564102564), ('Tuesday', 0.02564102564102564), ('Post', 0.02564102564102564)]
[]
[]
[]
